In [1]:
import json
import torch
import argparse
import numpy as np
from data import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from torch import optim
from torch.nn.functional import softmax
from torch.utils.data import DataLoader
from model import *
from utils import *
from random import randint, shuffle
from tqdm import tqdm
from train import getTrainSet

In [3]:
trainSet, label2IndexRumor, label2IndexStance, tfidf_vec = getTrainSet('../dataset/semeval2017-task8/')
#_, _, _ = getDevTestSet(args.dataPath, 'test', tfidf_vec)

inputDim = trainSet[0][0].size()[1]
trainSet = semEval2017Dataset(
    dataPath = '../dataset/semeval2017-task8/', 
    type = 'train'
)
trainLoader = DataLoader(trainSet, collate_fn=collate, shuffle=True)
testSet = semEval2017Dataset(
    dataPath = '../dataset/semeval2017-task8/',
    type = 'test'
)
testLoader = DataLoader(testSet, collate_fn=collate, shuffle=True)

In [7]:
for thread in trainLoader:
    print(len(thread['thread']))
    cropus = []
    for post in thread['thread']:
        cropus.append(post)
    tfidf_matrix = tfidf_vec.fit_transform(cropus)
    counter = 0
    for i in range(len(thread['thread'])):
        tfidf = []
        for _ in threads[i]:
            tfidf.append(tfidf_matrix[counter])
            counter += 1
    break

18
(18, 94)


In [6]:
from torch import optim
from torch.nn.functional import softmax
loss_func = torch.nn.CrossEntropyLoss(reduction='sum')
optimizer = optim.Adagrad(mtes.parameters(), lr=1e-3)

NameError: name 'mtes' is not defined

In [27]:
#rumor test
x = threads[0]
rumorTag = rumorTags[0]
stanceTag = stanceTags[0]
print(x.size(), rumorTag.size(), stanceTag.size())

optimizer.zero_grad()
p1 = mtes.forwardRumor(x)
print(p1.size(), rumorTag)
loss = loss_func(p1, rumorTag)
loss.backward()
optimizer.step()
p1 = softmax(p1, 1)
print(p1)
p1 = p1.max(dim=1)[1].item()
print(p1)

torch.Size([5, 6586]) torch.Size([1]) torch.Size([5])
torch.Size([5, 1, 200])
torch.Size([1, 3]) tensor([0])
tensor([[0.6789, 0.1548, 0.1664]], grad_fn=<SoftmaxBackward0>)
0


In [32]:
#stance test
optimizer.zero_grad()
p2 = mtes.forwardStance(x).view(-1, len(label2IndexStance))
print(p2.size(), stanceTag)
loss = loss_func(p2, stanceTag)
loss.backward()
optimizer.step()
p2 = softmax(p2, 1)
print(p2)
p2 = p2.max(dim=1)[1].tolist()
print(p2)

torch.Size([5, 4]) tensor([0, 3, 3, 3, 0])
tensor([[0.3533, 0.0561, 0.0703, 0.5203],
        [0.3062, 0.0456, 0.0546, 0.5936],
        [0.2968, 0.0414, 0.0484, 0.6134],
        [0.3320, 0.0412, 0.0468, 0.5800],
        [0.4296, 0.0452, 0.0508, 0.4744]], grad_fn=<SoftmaxBackward0>)
[3, 3, 3, 3, 3]


In [20]:
from sklearn.metrics import f1_score
import numpy as np
y_true = np.array([0,1,2,2,1,0])
y_pred = np.array([0,0,0,1,0,1])
(y_true == y_pred).sum()

1